# Previsão de Vendas de Sorvetes

## Importação de Bibliotecas

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError
import os
from dotenv import load_dotenv
import joblib

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configurar credenciais e cliente Azure ML
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_WORKSPACE_NAME")
)

# Nome do recurso de computação
compute_name = "compute-instance-dinamico"

# Criar o recurso de computação, se não existir
try:
    compute_instance = ml_client.compute.get(compute_name)
    print(f"Recurso de computação já existe: {compute_instance.name}")
except ResourceNotFoundError:
    print(f"Recurso de computação '{compute_name}' não encontrado. Criando um novo...")
    compute_instance = AmlCompute(
        name=compute_name,
        size="STANDARD_DS11_V2",
        min_instances=0,
        max_instances=1
    )
    compute_instance = ml_client.compute.begin_create_or_update(compute_instance).result()
    print(f"Recurso de computação criado: {compute_instance.name}")

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app '87fae7b9-85ee-4b9f-98a3-73c46f8fad1e'. Trace ID: a677e2e2-81f6-4900-bd5b-89a60bf93800 Correlation ID: cb3147cc-901a-4731-81ce-c040dcf8a088 Timestamp: 2025-04-09 21:56:32Z
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.


ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app '87fae7b9-85ee-4b9f-98a3-73c46f8fad1e'. Trace ID: a677e2e2-81f6-4900-bd5b-89a60bf93800 Correlation ID: cb3147cc-901a-4731-81ce-c040dcf8a088 Timestamp: 2025-04-09 21:56:32Z
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.

In [ ]:
!python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Python 3.12.1 (main, Mar 17 2025, 17:13:06) [GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
Traceback (most recent call last):
  File "<stdin>", line 0, in <module>
KeyboardInterrupt
>>> 

## Carregamento de Dados

In [ ]:
!python
sorvetes = pd.read_csv("../data/raw/vendas_sorvetes.csv")
sorvetes.head()

Python 3.12.1 (main, Mar 17 2025, 17:13:06) [GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
Traceback (most recent call last):
  File "<stdin>", line 0, in <module>
KeyboardInterrupt
>>> 

,Dia,Temperatura (°C),Vendas
0,01/01/2023,20,30
1,02/01/2023,22,35
2,03/01/2023,24,40
3,04/01/2023,26,50
4,05/01/2023,28,60


## Análise Exploratória

In [ ]:
!python
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sorvetes, x='Temperatura', y='Vendas')
plt.title('Relação entre Temperatura e Vendas de Sorvetes')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Vendas')
plt.show()

## Pré-processamento de Dados

In [ ]:
!python
# Separar características e rótulos
X = sorvetes[['Temperatura']]
y = sorvetes['Vendas']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
!python
# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

## Avaliação do Modelo

In [ ]:
!python
# Fazer previsões
y_pred = model.predict(X_test)

# Calcular métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

In [ ]:
import joblib
import os

# Criar o diretório 'data/processed' se não existir
os.makedirs("../data/processed", exist_ok=True)

# Salvar o modelo treinado
model_path = "../data/processed/modelo_regressao_linear.pkl"
joblib.dump(model, model_path)
print(f"Modelo salvo em: {model_path}")

In [ ]:
# Salvar as previsões e os dados reais em um arquivo CSV
resultados = pd.DataFrame({
    "Valores Reais": y_test.values,
    "Valores Previstos": y_pred
})
resultados_path = "../data/processed/resultados.csv"
resultados.to_csv(resultados_path, index=False)
print(f"Resultados salvos em: {resultados_path}")